<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=8b3546c67186e4b2c53e2121c38292f1d6faa658d42939ba98f44be5764c460d
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-11 14:47:01
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.39 C
-------------------
Today PnL: 47.88 K (0.34%)
Current PnL: -28.83 L (-18.62%)
CY Booked + Current PnL: -14.54 L (-9.39%)
-------------------
Total profit:  1.22 L
Total loss:  -30.05 L
-------------------
Total Booked + Current PnL: 12.44 L (9.73%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.25%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 93.46 L (67.02%)
Deployed:  1.28 C
Current:  1.39 C
CAGR/XIRR %: 4.88%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,1.16,-16.44,19.78,0.09,16657.0,-16564.0,84213.0,95.52,45.0,M-SC,2.37,253.0,-0.99,0.63,10.64,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.37,-12.55,14.37,0.02,21937.0,-21903.0,152656.0,132.70,52.0,M-SC,16.23,234.0,-1.00,1.14,32.78,OX40N,NTT,PAINTS
50,MASFIN,398.61,0.20,-2.02,24.57,22.05,23587.0,-1980.0,96000.0,-15.54,56.0,H-SC,7.10,164.0,-0.08,0.71,39.54,XR,ATH,FINANCE
43,ITC,452.00,-0.00,-1.50,12.12,10.44,23893.0,-2998.0,197140.0,-41.15,47.0,X-LC,1.75,5.0,-0.13,1.47,3.98,X40,NTT,FMCG
82,VOLTAS,1530.00,0.71,5.64,13.30,19.69,26940.0,10818.0,202560.0,-4.67,48.0,X-MC,2.56,78.0,0.40,1.51,12.82,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,4.13,-17.40,47.20,21.58,54634.0,-24390.0,115749.0,-58.39,27.0,X-MC,7.11,56.0,-0.45,0.86,4.13,X40N,ATH,IT
18,COFFEEDAY,80.00,3.72,-42.93,137.04,35.27,88802.0,-48749.0,64800.0,-55.73,37.0,L-SC,10.73,270.0,-0.55,0.48,58.60,XR,NTT,HOTELS
64,SHALBY,327.00,3.24,-24.56,65.82,25.10,89950.0,-44483.0,136660.0,904.78,32.0,M-SC,11.01,250.0,-0.49,1.02,9.52,XY24,NTT,HEALTHCARE
72,TATAELXSI,9161.00,2.89,-27.89,83.09,32.03,78990.0,-36769.0,95066.0,-21.91,39.0,H-SC,8.58,148.0,-0.47,0.71,6.48,OX40N,NTT,IT
10,BANDHANBNK,400.00,2.64,-30.21,175.73,92.42,341609.0,-84165.0,194394.0,36.49,43.0,H-SC,8.58,170.0,-0.25,1.45,12.34,XY24,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,CAMPUS,393.0,-2.20,-10.91,49.60,33.28,74140.0,-18299.0,149476.0,-28.28,39.0,M-SC,2.80,222.0,-0.25,1.11,15.85,XY24,NTT,FOOTWEAR
12,BERGEPAINT,680.0,-2.16,-4.29,26.57,21.14,57813.0,-9753.0,217586.0,-11.03,38.0,X-MC,4.16,74.0,-0.17,1.62,22.64,XY24,NTT,PAINTS
16,CERA,9475.0,-1.61,-31.10,79.81,23.89,96728.0,-54705.0,121198.0,-33.41,31.0,H-SC,9.75,157.0,-0.57,0.90,4.67,OX40N,NTT,CERAMICS
59,RELAXO,1176.0,-1.36,-47.41,189.58,52.31,144739.0,-68813.0,76347.0,-44.05,43.0,X-SC,6.43,91.0,-0.48,0.57,2.19,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.0,-1.33,-38.10,118.42,35.20,94318.0,-49023.0,79647.0,4.40,34.0,X-SC,15.68,92.0,-0.52,0.59,1.17,X40,NTT,FOOTWEAR


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.49,-6.65,122.73,107.91,169145.0,-9825.0,137819.0,-24.20,39.0,M-SC,12.35,216.0,-0.06,1.02,2.76,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.37,-12.55,14.37,0.02,21937.0,-21903.0,152656.0,132.70,52.0,M-SC,16.23,234.0,-1.00,1.14,32.78,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,1.16,-16.44,19.78,0.09,16657.0,-16564.0,84213.0,95.52,45.0,M-SC,2.37,253.0,-0.99,0.63,10.64,OX40N,NTT,DURABLES
66,SIS,528.00,-0.70,-25.41,62.56,21.26,51815.0,-28208.0,82824.0,1952.33,43.0,H-SC,4.61,166.0,-0.54,0.62,11.90,OX40N,NTT,MISC
47,KANSAINER,340.00,-0.66,-25.53,52.38,13.47,105188.0,-68850.0,200817.0,-69.48,30.0,H-SC,4.18,158.0,-0.65,1.49,2.11,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.22,6.05,62.44,72.27,112204.0,10251.0,179699.0,-8.8,66.0,M-LC,1.93,99.0,0.09,1.34,12.6,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.22,6.05,62.44,72.27,112204.0,10251.0,179699.0,-8.80,66.0,M-LC,1.93,99.0,0.09,1.34,12.60,XR,NTT,IT
38,INDIAMART,4810.62,0.24,-3.74,114.76,106.72,136243.0,-4616.0,118720.0,-52.34,32.0,H-SC,1.99,139.0,-0.03,0.88,18.76,AR,ATH,MISC
25,FINCABLES,1641.55,-0.49,-6.65,122.73,107.91,169145.0,-9825.0,137819.0,-24.20,39.0,M-SC,12.35,216.0,-0.06,1.02,2.76,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,-0.26,-2.86,39.62,35.63,80660.0,-5989.0,203585.0,-16.74,37.0,H-MC,3.16,119.0,-0.07,1.51,13.34,AR,ATH,PHARMA
37,IEX,219.00,1.84,-3.61,54.19,48.63,104443.0,-7219.0,192735.0,-35.77,50.0,H-SC,14.05,136.0,-0.07,1.43,7.53,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.39,-21.60,89.36,48.45,140469.0,-43312.0,157194.0,-27.18,25.0,X-MC,12.19,64.0,-0.31,1.17,2.35,X40N,ATH,FINANCE
22,DIXON,18931.72,4.13,-17.40,47.20,21.58,54634.0,-24390.0,115749.0,-58.39,27.0,X-MC,7.11,56.0,-0.45,0.86,4.13,X40N,ATH,IT
33,HINDUNILVR,2922.00,-0.07,-10.52,27.03,13.67,62174.0,-27043.0,230020.0,-17.28,28.0,X-LC,2.63,14.0,-0.43,1.71,8.59,XY25,NTT,FMCG
75,TMPV,600.00,0.95,-17.86,73.09,42.18,165955.0,-49354.0,227056.0,-26.74,29.0,X-LC,6.04,3.0,-0.30,1.69,0.95,XY24,NTT,AUTO
23,DMART,5391.45,0.15,-9.18,41.07,28.12,78483.0,-19312.0,191095.0,-21.67,31.0,X-LC,4.95,19.0,-0.25,1.42,12.28,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.0,1.98,-80.30,533.41,24.79,264673.0,-202237.0,49619.0,-81.32,43.0,X-SC,1.24,86.0,-0.76,0.37,21.30,X40N,NTT,MISC
8,AWL,485.0,-0.00,-21.78,96.24,53.50,227153.0,-65723.0,236028.0,-62.10,33.0,X-MC,1.71,58.0,-0.29,1.75,4.84,XY24,NTT,FMCG
43,ITC,452.0,-0.00,-1.50,12.12,10.44,23893.0,-2998.0,197140.0,-41.15,47.0,X-LC,1.75,5.0,-0.13,1.47,3.98,X40,NTT,FMCG
36,ICICIPRULI,790.0,-1.25,5.24,24.45,30.97,45476.0,9258.0,185996.0,-19.54,59.0,X-MC,2.01,75.0,0.20,1.38,18.46,X40,ATH,INSURANCE
3,ACC,3906.0,-0.20,-25.25,119.60,64.15,212733.0,-60081.0,177870.0,-56.31,34.0,X-SC,2.11,82.0,-0.28,1.32,0.00,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.20,-25.25,119.60,64.15,212733.0,-60081.0,177870.0,-56.31,34.0,X-SC,2.11,82.0,-0.28,1.32,0.00,XY24,BTT,CEMENT
54,PGHH,17907.65,-0.40,-6.26,42.69,33.76,80364.0,-12570.0,188250.0,-33.85,32.0,X-MC,5.24,57.0,-0.16,1.40,0.00,X40,ATH,FMCG
31,HAVELLS,2069.16,0.60,-10.48,47.99,32.48,140909.0,-34369.0,293622.0,-15.05,37.0,X-MC,5.50,67.0,-0.24,2.18,0.60,X40,ATH,ELECTRICAL
75,TMPV,600.00,0.95,-17.86,73.09,42.18,165955.0,-49354.0,227056.0,-26.74,29.0,X-LC,6.04,3.0,-0.30,1.69,0.95,XY24,NTT,AUTO
53,PAGEIND,51605.08,1.05,-9.42,39.17,26.07,58097.0,-15420.0,148320.0,-33.27,33.0,X-MC,12.13,55.0,-0.27,1.10,1.05,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,0.12,-29.73,103.27,42.84,47175.0,-19325.0,45681.0,-53.72,42.0,X-SC,36.52,83.0,-0.41,0.34,3.35,XY24,NTT,MISC
15,CAMS,4762.00,1.98,-80.30,533.41,24.79,264673.0,-202237.0,49619.0,-81.32,43.0,X-SC,1.24,86.0,-0.76,0.37,21.30,X40N,NTT,MISC
59,RELAXO,1176.00,-1.36,-47.41,189.58,52.31,144739.0,-68813.0,76347.0,-44.05,43.0,X-SC,6.43,91.0,-0.48,0.57,2.19,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.33,-38.10,118.42,35.20,94318.0,-49023.0,79647.0,4.40,34.0,X-SC,15.68,92.0,-0.52,0.59,1.17,X40,NTT,FOOTWEAR
22,DIXON,18931.72,4.13,-17.40,47.20,21.58,54634.0,-24390.0,115749.0,-58.39,27.0,X-MC,7.11,56.0,-0.45,0.86,4.13,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,-0.05,-13.39,37.72,19.28,113026.0,-46312.0,299644.0,-24.60,59.0,X-LC,5.21,1.0,-0.41,2.23,10.77,X40,ATH,IT
41,INFY,2275.00,0.52,8.45,42.80,54.87,145233.0,26442.0,339330.0,-15.65,61.0,X-LC,3.22,2.0,0.18,2.52,17.43,X40,BTT,IT
75,TMPV,600.00,0.95,-17.86,73.09,42.18,165955.0,-49354.0,227056.0,-26.74,29.0,X-LC,6.04,3.0,-0.30,1.69,0.95,XY24,NTT,AUTO
81,VBL,671.64,0.70,-3.80,41.03,35.67,124669.0,-11994.0,303848.0,-15.30,56.0,X-LC,4.59,4.0,-0.10,2.26,9.53,X40N,ATH,FMCG
43,ITC,452.00,-0.00,-1.50,12.12,10.44,23893.0,-2998.0,197140.0,-41.15,47.0,X-LC,1.75,5.0,-0.13,1.47,3.98,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,1.42,-16.62,117.88,81.67,92209.0,-15587.0,78223.0,7027.78,44.0,L-SC,13.93,271.0,-0.17,0.58,51.48,XR,NTT,CERAMICS
50,MASFIN,398.61,0.20,-2.02,24.57,22.05,23587.0,-1980.0,96000.0,-15.54,56.0,H-SC,7.10,164.0,-0.08,0.71,39.54,XR,ATH,FINANCE
13,BSOFT,831.70,1.23,-22.35,94.19,50.79,102871.0,-31437.0,109216.0,-0.27,67.0,H-SC,5.86,151.0,-0.31,0.81,27.16,XR,ATH,IT
39,INDIGOPNTS,1408.00,0.37,-12.55,14.37,0.02,21937.0,-21903.0,152656.0,132.70,52.0,M-SC,16.23,234.0,-1.00,1.14,32.78,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.70,-2.66,24.25,20.95,59430.0,-6697.0,245071.0,-3.58,47.0,X-LC,18.67,31.0,-0.11,1.82,32.03,X40,ATH,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,1.23,-22.35,94.19,50.79,102871.0,-31437.0,109216.0,-0.27,67.0,H-SC,5.86,151.0,-0.31,0.81,27.16,XR,ATH,IT
56,RAJESHEXPO,518.00,0.91,-62.92,169.85,0.07,86401.0,-86308.0,50869.0,1764.90,57.0,L-SC,2.38,268.0,-1.00,0.38,26.51,OX40N,NTT,JEWELLERY
84,WIPRO,420.00,0.22,6.05,62.44,72.27,112204.0,10251.0,179699.0,-8.80,66.0,M-LC,1.93,99.0,0.09,1.34,12.60,XR,NTT,IT
32,HCLTECH,1922.01,0.09,8.21,15.26,24.73,39950.0,19874.0,261798.0,13.18,64.0,X-LC,6.20,7.0,0.50,1.95,24.70,X40,ATH,IT
0,5PAISA,593.00,1.48,-41.43,92.75,12.90,118703.0,-90514.0,127982.0,88.70,43.0,H-SC,11.65,161.0,-0.76,0.95,6.42,OX40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.74
1,25,45.70
2,50,76.66


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.10
MC    30.23
LC    24.66
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.29
X40      22.43
X40N     10.82
XR       10.14
XY25      9.39
AR        9.34
OX40N     7.81
SR        1.02
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.57
X-MC    23.49
X-LC    21.08
M-SC    12.15
X-SC     6.94
H-MC     4.99
L-SC     1.44
M-MC     1.40
M-LC     1.34
H-LC     1.18
L-LC     1.06
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.38,-7.69,42.52
IT,12.77,-17.38,80.84
FINANCE,9.58,-20.28,71.11
ELECTRICAL,6.22,-11.90,52.90
MISC,6.10,-58.23,123.05
PAINTS,6.07,-13.14,29.94
INSURANCE,4.60,-2.12,38.11
PHARMA,4.11,-1.11,33.47
AUTO,3.42,-25.20,78.89


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3232230.0,21
AR,1313189.0,10
XR,1296548.0,13
X40,1022397.0,14
X40N,993169.0,9
OX40N,743824.0,10
XY25,378755.0,6
SR,282841.0,2
MH,83075.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3672740.0,25
M-SC,1426204.0,15
X-MC,1323703.0,16
X-SC,985785.0,8
X-LC,927849.0,11
H-MC,406624.0,3
L-SC,267412.0,3
M-LC,112204.0,1
H-LC,74313.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1285732.0      6
           AR         900364.0      5
M-SC       XY24       813572.0      6
H-SC       XR         774492.0      7
X-SC       X40N       507687.0      3
X-MC       X40        458539.0      7
           XY24       398326.0      3
X-LC       X40        381532.0      5
H-SC       OX40N      346236.0      4
M-SC       OX40N      311187.0      5
X-SC       XY24       295772.0      3
H-SC       SR         282841.0      2
X-MC       X40N       282330.0      4
X-LC       XY24       243017.0      2
H-MC       AR         210813.0      2
X-LC       X40N       203152.0      2
H-MC       XY24       195811.0      1
X-MC       XY25       184508.0      2
X-SC       X40        182326.0      2
L-SC       XR         181011.0      2
M-SC       XR         173746.0      2
           AR         127699.0      2
M-LC       XR         112204.0      1
X-LC       XY25       100148.0      2
L-SC       OX40N       86401.0      1
H-SC       MH          83075.0      1
H-LC       AR          74313.0      1
M-MC       XY25        57015.0      1
L-MC       XR          55095.0      1
L-LC       XY25        37084.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 56.0 seconds
